In [1]:
from Bio import SeqIO
import os
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from plotly.offline import iplot, init_notebook_mode

import pandas as pd
pd.set_option('display.max_colwidth', 0)

import plotly.plotly as py
import plotly.graph_objs as go
from ipywidgets import HTML
from ipywidgets import HBox, VBox


############################# put your antismash 5 gbk output in here #############################

#DATA import
gb = 'Azum.gbk'

################################################################################################





##Lists and dictionaries 

##Parse gbk to lists
locos = []
for record in SeqIO.parse(gb, "gb"):
     for feature in record.features:
        if feature.type == 'CDS':
            locos.append(feature.location)
            


            
#for mapping tick marks 
x = [x for x in range(0,locos[-1].end, 1000)]+[locos[-1].end]



##For mapping gene type colors 
gene_colors = {'biosynthetic': 'rgb(126, 16, 25)',
               'regulatory': 'rgb(50,138,89)',
               'biosynthetic-additional':'rgb(238,110,119)' ,
               'resistance': 'rgb(235,47,235)',
               'transport': 'rgb(102, 151, 234)',
               'N':'rgb(128,128,128)',
               'other':'rgb(101,202,56)'
}


##For mapping points to orfs  
orf_points = [(location.start + location.end)/2 for location in locos]



all_features = []
for record in SeqIO.parse(gb, "gb"):
    for feature in record.features:
        if feature.type == 'CDS':
            all_features.append(feature)
            

names = [feature.qualifiers.get('gene')[0] for feature in all_features if feature.qualifiers.get('gene')]
if not names:
    all_locus_tags = [feature.qualifiers.get('locus_tag')[0] for feature in all_features if feature.qualifiers.get('locus_tag')] 
    for tag in all_locus_tags:
        if tag not in names:
            names.append(tag)
            
p_strings = [feature.qualifiers['translation'][0] for feature in all_features]

#write the seqs to a multifasta file temp.faa
for_fasta = [SeqRecord(Seq(seq_string, IUPAC.protein), id=seq_id) for seq_string, seq_id  in zip(p_strings, names)]
SeqIO.write(for_fasta, f"{gb}.faa", 'fasta')





49

## Now you need to do the BLASTp on the *.gbk.faa file, that was just generated, using the webserver and DL the xml to this directory and run the next cell.

In [2]:
blast_xml = f"{gb}.xml"

blast_records = NCBIXML.parse(open(blast_xml))
data_dict = {}

for blast_record in blast_records:
    
    data = []
    for alignment in blast_record.alignments: 
        data.append(alignment.title.split('|')[2].replace('>gb',''))
        
        for hsp in alignment.hsps:            
            data.append(round(((hsp.identities/hsp.align_length)*100),2))
            data_dict[blast_record.query]=data
            break
        
        break
         

            
gene_types = [feature.qualifiers.get('gene_kind', [None])[0] for feature in all_features if feature.type == 'CDS']
            
blast_table = pd.DataFrame.from_dict(data_dict, orient='index',columns=['Top Blast Hit','% identity'])   
blast_table['gene_type'] = gene_types



##Functions
##To get gene colors, needs to be run through AS v5. 
def gene_type_color(feature):
    gene_type = feature.qualifiers.get('gene_kind', 'None')
    
    return gene_colors[gene_type[0]]


def frd_to_svg(feature):
    
    if len(feature.location) < locos[-1].end/100:
        nose_offset = len(feature.location)
    else:
        nose_offset = locos[-1].end/100
    
    
    start = feature.location.start
    length = (feature.location.start + len(feature.location)) - nose_offset
    nose = length + nose_offset 
    
    
    return f'M {start},0.1 L{start},1.1 L{length},1.1 L{nose},.6 L{length},0.1 L {start},0.1'


def rev_to_svg(feature):
    
    if len(feature.location) < locos[-1].end/100:
        nose_offset = len(feature.location)
    else:
        nose_offset = locos[-1].end/100
    
    
    start = feature.location.start + nose_offset 
    length = (feature.location.start + len(feature.location)) 
    nose = start - nose_offset 
    
    
    return f'M {start},0.1 L{length},0.1 L{length},1.1 L{start},1.1 L{nose},0.6 L {start},0.1'




plot_range = locos[-1].end ## total length of plot to show (gene cluster length plus 5%) 

##List of all the + strand gens to show on the plot
            
frd_orfs = [{'type': 'path',
            'path': frd_to_svg(feature),
            
           'fillcolor': gene_type_color(feature), #make funct for fill color (fill_col(feature_location))
            
           'line': {'color': gene_type_color(feature),
                    'width': 1.2}
                    } 
             
            for feature in all_features if feature.location.strand == 1]   

rev_orfs = [{'type': 'path',
            'path': rev_to_svg(feature),
            
           'fillcolor': gene_type_color(feature), #make funct for fill color (fill_col(feature_location))
            
           'line': {'color': gene_type_color(feature),
                   'width': 1.2}
                    } 
             
            for feature in all_features if feature.location.strand == -1]   




trace = go.Scatter(
    x = orf_points,
    y = [.5]*len(orf_points),
    mode = 'markers',
    hoverinfo = 'text',
    text = names
)


data = [trace]

layout={'xaxis': {
        'showgrid':False,
        'zeroline':False,
        'showline':False,
        'tickmode':'linear',
        'ticks':'',
        'showticklabels':False,
        'range': [0 , plot_range ]},
        
        'yaxis':{
        'range': [-4 ,4],
        'showgrid':False,
        'zeroline':True,
        'showline':False,
        'ticks':'',
        'showticklabels':False},
         
        'shapes': frd_orfs + rev_orfs
       }


fig = go.FigureWidget(data,layout)


#updates to figures if using ipywidgets

fig.layout.title = 'Gene cluster'

scatter = fig.data[0]
scatter.marker.size = 1

details = HTML()

def hov_func(trace, points, state):
    ind = points.point_inds[0]
    details.value = blast_table.iloc[ind].to_frame().to_html()

scatter.on_hover(hov_func)

VBox([fig, HBox([details])])

    'data': [{'hoverinfo': 'text',
              'marker': {'size': 1},
        …